In [1]:
import xarray as xr
import os
import sys
import dask.distributed
import glob
from dask.distributed import Client
import tempfile
import dask
import numpy as np
import time
import matplotlib.pyplot as plt

sys.path.append('/home/565/dh4185/mn51-dh4185/repos_collab/dataset_finder/')
from dataset_finder import *

In [4]:
# Dask settings
dask.config.set({
    #'array.chunk-size': "256 MiB",
    #'array.slicing.split_large_chunks': True, 
    'distributed.comm.timeouts.connect': '120s',
    'distributed.comm.timeouts.tcp': '120s',
    'distributed.comm.retry.count': 10,
    'distributed.scheduler.allowed-failures': 20,
    "distributed.scheduler.worker-saturation": 1.1, #< This should use the new behaviour which helps with memory pile up
})

client = Client(n_workers=25, threads_per_worker=1, local_directory = tempfile.mkdtemp(), memory_limit = "32000mb")
client

/g/data/xp65/public/apps/med_conda/envs/analysis3-25.07/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44295 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/44295/status,
Dashboard: /proxy/44295/status,Workers: 25
Total threads: 25,Total memory: 745.06 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46075,Workers: 0
Dashboard: /proxy/44295/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:36187,Total threads: 1
Dashboard: /proxy/41045/status,Memory: 29.80 GiB
Nanny: tcp://127.0.0.1:37683,


In [5]:
####< Functions
# Function to compute great-circle distance using NumPy (vectorised)
def great_circle_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth's radius in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    return R * c  # Distance in km

def update_locations(da_sftlf,dict_locations):
    ### Find nearest grid cell on land to lat/lon coordinates from stations
    # Extract grid information
    latitudes = da_sftlf['lat'].values
    longitudes = da_sftlf['lon'].values
    valid_mask = da_sftlf.values >= 90  # Mask where terrain fraction is at least 90
    
    # Get valid lat/lon pairs
    valid_lats, valid_lons = np.meshgrid(latitudes, longitudes, indexing='ij')  # Ensure correct shape
    valid_lats, valid_lons = valid_lats[valid_mask], valid_lons[valid_mask]  # Apply mask
    
    # Find the closest valid grid cell for each location & store in a new dictionary
    updated_locations = {}
    for name, info in dict_locations.items():
        lat, lon, elev = info['Lat'], info['Lon'], info['Elev']
        distances = great_circle_distance(lat, lon, valid_lats, valid_lons)
        
        min_idx = np.argmin(distances)  # Index of closest point
        new_lat, new_lon = valid_lats[min_idx], valid_lons[min_idx]
        
        updated_locations[name] = {'Lat': new_lat, 'Lon': new_lon, 'Elev': elev}  # Preserve elevation
        
    return updated_locations
             
def preprocess_location(ds,lat,lon):
    return ds.sel(lat=lat,lon=lon, method="nearest")
             # .sel(time=slice(f'{start_year}-01-01', f'{end_year}-12-31'))

In [6]:
locations = {'Darwin':{'Lat':-12.42,'Lon':130.89,'Elev':30}, # BOM station: 014015
             'Cairns':{'Lat':-16.87,'Lon':145.75,'Elev':2}, # BOM station: 031011
             'Brisbane':{'Lat':-27.48,'Lon':153.04,'Elev':8}, # BOM station: 040913
             'Longreach':{'Lat':-23.44,'Lon':144.28,'Elev':192}, # BOM station: 036031
             'Mildura':{'Lat':-34.23,'Lon':142.09,'Elev':50}, # BOM station: 076031
             'Adelaide':{'Lat':-34.95,'Lon':138.51,'Elev':2}, # BOM station: 023034
             'Perth':{'Lat':-31.92,'Lon':115.87,'Elev':25}, # BOM station: 009225
             'Sydney':{'Lat':-33.95,'Lon':151.17,'Elev':39}, # BOM stations: 066062/066214
             'Melbourne':{'Lat':-37.67,'Lon':144.83,'Elev':31}, # BOM stations: 086071/086338
             'Canberra':{'Lat':-35.31,'Lon':149.19,'Elev':577}, # BOM station: 070351
             'Hobart':{'Lat':-42.89,'Lon':147.33,'Elev':51}, # BOM station: 094029
             'Thredbo':{'Lat':-36.50,'Lon':148.29,'Elev':1380}} # BOM station: 071041

In [7]:
#parameters
out_dir = "/home/565/dh4185/mn51-dh4185/repos_collab/nesp_bff/data/future_heat/"

## Tasmax from AGCDv2

In [7]:
%%time
#< Specify datasets - do this to find out what models have the required variables
data_day_agcd = get_datasets("AGCD",
                        # gcm = _gcm
                        timescale = "day",
                        var = "tmax",
                        year = year_range(1995, 2014))

# data_day_agcd = data_day_agcd.select(var = "tmax", exact_match=True)
data_day_agcd

da = xr.open_mfdataset(data_day_agcd.get_files(), parallel=True).tmax
da

CPU times: user 7.04 s, sys: 1.36 s, total: 8.4 s
Wall time: 36.4 s


<xarray.DataArray 'tmax' (time: 7305, lat: 691, lon: 886)> Size: 18GB
dask.array<concatenate, shape=(7305, 691, 886), dtype=float32, chunksize=(1, 691, 886), chunktype=numpy.ndarray>
Coordinates:
  * lon      (lon) float32 4kB 112.0 112.1 112.1 112.2 ... 156.1 156.2 156.2
  * lat      (lat) float32 3kB -44.5 -44.45 -44.4 -44.35 ... -10.1 -10.05 -10.0
  * time     (time) datetime64[ns] 58kB 1995-01-01T09:00:00 ... 2014-12-31T09...
Attributes:
    standard_name:                 air_temperature
    long_name:                     Daily maximum air temperature
    units:                         degrees_Celsius
    cell_methods:                  time: maximum
    analysis_version_number:       3.51
    number_of_stations_reporting:  584
    source:                        Interpolated surface observations
    frequency:                     daily
    length_scale_for_analysis:     250.0
    analysis_time:                 2010-10-07 02:58:27 UTC

In [9]:
%%time
ds_ge30 = (da.where(da>=30).groupby('time.month').count(dim='time')/20).rename("tmax_ge30").astype("float32").to_dataset()
ds_ge35 = (da.where(da>=35).groupby('time.month').count(dim='time')/20).rename("tmax_ge35").astype("float32").to_dataset()
# ds_ge40 = (da.where(da>=40).groupby('time.month').count(dim='time')/20).rename("tmax_ge40").astype("float32").to_dataset()

ds_out = xr.merge([ds_ge30,ds_ge35])#,ds_ge40])
print(ds_out)
ds_out.to_netcdf(f"{out_dir}AGCDv2_heat-threshold-days_1995-2014.nc")

<xarray.Dataset> Size: 59MB
Dimensions:    (lon: 886, lat: 691, month: 12)
Coordinates:
  * lon        (lon) float32 4kB 112.0 112.1 112.1 112.2 ... 156.1 156.2 156.2
  * lat        (lat) float32 3kB -44.5 -44.45 -44.4 ... -10.1 -10.05 -10.0
  * month      (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    tmax_ge30  (month, lat, lon) float32 29MB dask.array<chunksize=(1, 691, 886), meta=np.ndarray>
    tmax_ge35  (month, lat, lon) float32 29MB dask.array<chunksize=(1, 691, 886), meta=np.ndarray>
CPU times: user 56.6 s, sys: 5.65 s, total: 1min 2s
Wall time: 2min 17s


## Tasmax from BARRA-R2

In [11]:
%%time
da = xr.open_mfdataset("/g/data/ia39/australian-climate-service/release/CORDEX/output-Adjust/CMIP6/bias-adjusted-input/AUST-05i/BOM/ERA5/historical/hres/BARRAR2/v1/day/tasmax/v20241216/tasmax*.nc", parallel=True).sel(time=slice('1995','2014')).tasmax
da

CPU times: user 367 ms, sys: 27.2 ms, total: 394 ms
Wall time: 538 ms


<xarray.DataArray 'tasmax' (time: 7305, lat: 691, lon: 886)> Size: 18GB
dask.array<getitem, shape=(7305, 691, 886), dtype=float32, chunksize=(1, 691, 886), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 58kB 1995-01-01T12:00:00 ... 2014-12-31T12...
  * lat      (lat) float64 6kB -44.5 -44.45 -44.4 -44.35 ... -10.1 -10.05 -10.0
  * lon      (lon) float64 7kB 112.0 112.0 112.1 112.2 ... 156.2 156.2 156.2
Attributes:
    long_name:      Daily Maximum Near-Surface Air Temperature
    standard_name:  air_temperature
    units:          degC
    regrid_method:  bilinear

In [12]:
%%time
da_ge30 = (da.where(da>=30).groupby('time.month').count(dim='time')/20).rename("tasmax_ge30").astype("float32").to_dataset()
da_ge35 = (da.where(da>=35).groupby('time.month').count(dim='time')/20).rename("tasmax_ge35").astype("float32").to_dataset()
# da_ge40 = (da.where(da>=40).groupby('time.month').count(dim='time')/20).rename("tasmax_ge40").astype("float32").to_dataset()

ds_out = xr.merge([da_ge30,da_ge35])#,compat='override')
ds_out

CPU times: user 147 ms, sys: 11.2 ms, total: 158 ms
Wall time: 150 ms


<xarray.Dataset> Size: 59MB
Dimensions:      (lat: 691, lon: 886, month: 12)
Coordinates:
  * lat          (lat) float64 6kB -44.5 -44.45 -44.4 ... -10.1 -10.05 -10.0
  * lon          (lon) float64 7kB 112.0 112.0 112.1 112.2 ... 156.2 156.2 156.2
  * month        (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    tasmax_ge30  (month, lat, lon) float32 29MB dask.array<chunksize=(1, 691, 886), meta=np.ndarray>
    tasmax_ge35  (month, lat, lon) float32 29MB dask.array<chunksize=(1, 691, 886), meta=np.ndarray>

In [13]:
ds_out.to_netcdf(f"{out_dir}BARRA-R2_heat-threshold-days_1995-2014.nc")

## Tasmax from BARPA-R and CSIRO-CCAM

In [8]:
periods = {"historical":{"start":1995,"end":2014},
           "ssp370":{"start":2041,"end":2060},
           "ssp126":{"start":2041,"end":2060},
           "ssp370":{"start":2080,"end":2099},
           "ssp126":{"start":2080,"end":2099}
          }
bc_ref = "AGCDv1"
bc_method = "QME"
           
models = {"BARPA-R":["ACCESS-ESM1-5","ACCESS-CM2","CESM2","CMCC-ESM2","EC-Earth3","MPI-ESM1-2-HR","NorESM2-MM"],
        "CCAM-v2203-SN":["ACCESS-ESM1-5","ACCESS-CM2","CESM2","CMCC-ESM2","CNRM-ESM2-1","EC-Earth3","NorESM2-MM"]
       }

In [ ]:
%%time

for _rcm in models:
    print(f"---------- {_rcm} -----------")
    for _gcm in models[_rcm]:
        print(_gcm)

        for _scenario in periods:
            start_time_loc = time.time()  # Start timer

            out_file = f"{out_dir}{_rcm}_{_gcm}_heat-threshold-days_QME-{bc_ref}_{_scenario}_{periods[_scenario]['start']}-{periods[_scenario]['end']}.nc"
            
            if not os.path.exists(out_file):
                
                print(f"{_scenario}: {periods[_scenario]['start']}-{periods[_scenario]['end']}")
            
                #< Specify datasets - do this to find out what models have the required variables
                data_day = get_datasets("ACS_BC",
                                        rcm = _rcm,
                                        gcm = _gcm,
                                        scenario = _scenario,
                                        timescale = "day",
                                        var = "tasmaxAdjust",
                                        bc = bc_method,
                                        ref = bc_ref,
                                        year = year_range(periods[_scenario]['start'], periods[_scenario]['end']))
                
                # data_day = data_day.select(var = "tasmax", exact_match=True)
                print(data_day)
            
                ds = xr.open_mfdataset(data_day.get_files(), parallel=True).tasmaxAdjust
            
                # get thresholds
                ds_ge30 = (ds.where(ds>=30).groupby('time.month').count(dim='time')/20).rename("tasmax_ge30").astype("float32").to_dataset()
                ds_ge35 = (ds.where(ds>=35).groupby('time.month').count(dim='time')/20).rename("tasmax_ge35").astype("float32").to_dataset()
                # ds_ge40 = (ds.where(ds>=40).groupby('time.month').count(dim='time')/20).rename("tasmax_ge40").astype("float32").to_dataset()
            
                ds_out = xr.merge([ds_ge30,ds_ge35])#,ds_ge40])
                print(ds_out)
                ds_out.to_netcdf(out_file)
                print(f"Processing time: {((time.time() - start_time_loc)/60):.2f} minutes\n")

            else:
                print(f'File {out_file} exists in output directory.')

    

---------- BARPA-R -----------
ACCESS-ESM1-5
File /home/565/dh4185/mn51-dh4185/repos_collab/nesp_bff/data/future_heat/BARPA-R_ACCESS-ESM1-5_heat-threshold-days_QME-AGCDv1_historical_1995-2014.nc exists in output directory.
File /home/565/dh4185/mn51-dh4185/repos_collab/nesp_bff/data/future_heat/BARPA-R_ACCESS-ESM1-5_heat-threshold-days_QME-AGCDv1_ssp370_2080-2099.nc exists in output directory.
File /home/565/dh4185/mn51-dh4185/repos_collab/nesp_bff/data/future_heat/BARPA-R_ACCESS-ESM1-5_heat-threshold-days_QME-AGCDv1_ssp126_2080-2099.nc exists in output directory.
ACCESS-CM2
File /home/565/dh4185/mn51-dh4185/repos_collab/nesp_bff/data/future_heat/BARPA-R_ACCESS-CM2_heat-threshold-days_QME-AGCDv1_historical_1995-2014.nc exists in output directory.
File /home/565/dh4185/mn51-dh4185/repos_collab/nesp_bff/data/future_heat/BARPA-R_ACCESS-CM2_heat-threshold-days_QME-AGCDv1_ssp370_2080-2099.nc exists in output directory.
File /home/565/dh4185/mn51-dh4185/repos_collab/nesp_bff/data/future_heat